In [100]:
from ollama import generate, chat, pull
from pydantic import BaseModel
from typing import List

pull('gemma3:12b')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [68]:
class QnA(BaseModel):
    question: str
    answer: bool
    explanation: str

class Response(BaseModel):
    responses: List[QnA] = []

In [101]:

response = generate(
    system='''
    Given a top view image or street view images, you are going to roughly estimate house conditions. 
    Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes, no, maybe), explanation (within 50 words) for each question.
    ''',
    prompt='Is there any damage on the roof?',
    model='gemma3:12b',
    images=['docs/data/test1.jpg'],
    format=Response.model_json_schema(),
    options={
                "temperature":0,
                "top_k":0.8,
                "top_p":0.8
            }
)

# Parse response content into the Pydantic model
# res = Response.model_validate_json(response.message.content)  # Note: use ['response'] for generate()
# res


In [63]:
response.response

"\nBased on the image provided, it is not possible to conclusively determine if there is any damage on the roof due to the lack of detail and resolution. The image does not allow for a clear view of the roof's condition, which would be necessary to provide an accurate assessment. A closer inspection or additional images from different angles would be required to accurately answer this question."

In [30]:
res = Response.model_validate_json(response.response)

In [50]:
res

Response(responses=[QnA(question='Is there any damage on the roof?', answer='no', explanation='The roof appears to be intact, with no visible signs of damage or wear. The shingles are evenly spaced and securely fastened.')])

In [104]:
system='''
    Given a top view image or street view images, you are going to roughly estimate house conditions. 
    Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes=true, no=false), explanation (within 50 words) for each question.
    '''
prompt='Is there any physical damage on the roof?'
response = chat(
    
    model='gemma3:12b',
    messages=[
                {
                    'role': 'system',
                    'content': system
                },
                {
                    'role': 'user',
                    'content': prompt,
                    'images': ['docs/data/test3.jpg']
                }
            ],
    format=Response.model_json_schema(),
    options={
                "temperature":0,
                "top_k":0.8,
                "top_p":0.8
            }
)

res2 = Response.model_validate_json(response.message.content)

In [105]:
res2

Response(responses=[QnA(question='Is there any physical damage on the roof?', answer=False, explanation='Based on the image, the roof appears to be intact and without any visible signs of damage like missing shingles or structural issues. It looks generally well-maintained.')])

In [107]:
import urbanworm

In [108]:
from urbanworm import UrbanDataSet

In [109]:
bbox = (-83.235572,42.348092,-83.235154,42.348806)
data = UrbanDataSet()
data.bbox2Buildings(bbox)

'5 buildings found in the bounding box.'

In [110]:
system = '''
    Given a top view image, you are going to roughly estimate house conditions. Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes=1 or no=0), explaination (within 50 words)
'''
prompt = {
    'top':'''
        Is there any damage on the roof?
    '''
}

# inspect the aerial images only (with type='top')
res = data.loopUnitChat(system=system, prompt=prompt, type='top', epsg=2253, model='gemma3:12b')

python(61872) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Processing...: 100%|█████████████████████████| 5/5 [01:40<00:00, 20.12s/it]


In [111]:
res

{'lon': [-83.23528425002904,
  -83.23527843983697,
  -83.23527392220824,
  -83.235269271178,
  -83.23526440263088],
 'lat': [42.34874838775614,
  42.34859720064423,
  42.34845229151653,
  42.34830635645473,
  42.34816433680549],
 'top_view': [[QnA(question='Is there any damage on the roof?', answer='0', explanation="The roof appears intact and doesn't show any obvious signs of damage like missing shingles or structural issues. The image quality is limited, but no immediate concerns are visible.")],
  [QnA(question='Is there any damage on the roof?', answer='0', explanation='The roof appears relatively intact from this top-down view. There are no obvious signs of missing shingles, holes, or other structural damage visible.')],
  [QnA(question='Is there any damage on the roof?', answer='0', explanation="Based on the image, the roof appears intact and doesn't show any obvious signs of damage like missing shingles or structural issues. The resolution is limited, but no immediate concerns a

In [112]:
df = data.to_gdf()

In [113]:
df

,geometry,top_view_question1,top_view_answer1,top_view_explanation1,top_view_base64
0,POINT (-83.23528 42.34875),Is there any damage on the roof?,0,The roof appears intact and doesn't show any o...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,POINT (-83.23528 42.3486),Is there any damage on the roof?,0,The roof appears relatively intact from this t...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,POINT (-83.23527 42.34845),Is there any damage on the roof?,0,"Based on the image, the roof appears intact an...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,POINT (-83.23527 42.34831),Is there any damage on the roof?,0,The roof appears to be flat and uniform. There...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,POINT (-83.23526 42.34816),Is there any damage on the roof?,0,The roof appears intact and uniform. There are...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
def chatpd(df, prompt, output_type='string', model='gemma3:12b'):
    from pandasai import PandasAI
    from pandasai.llm.base import LLM
    import requests

    UrbanDataSet.preload_model(model)
    class OllamaLLM(LLM):
        def __init__(self, model=model, host='http://localhost:11434'):
            self.model = model
            self.host = host

        def call(self, prompt: str, *args, **kwargs) -> str:
            prompt = str(prompt)  # ensure it's a string

            messages = [
                {"role": "system", "content": "You are a helpful data science assistant."},
                {"role": "user", "content": prompt}
            ]

            response = requests.post(
                f"{self.host}/api/chat",
                json={
                    "model": self.model,
                    "messages": messages,
                    "stream": False
                }
            )
            response.raise_for_status()
            return response.json()["message"]["content"]

        @property
        def type(self) -> str:
            return "ollama"
    llm = OllamaLLM(model=model)
    pandas_ai = PandasAI(llm)
    df.chat = lambda q: pandas_ai.run(df, q)
    df = SmartDataframe(df, config={"llm": llm})
    return df.chat(prompt, output_type)

In [137]:
chatpd(df, 'please summarize the data')

/var/folders/fb/4kj6xrcs195bxml3gxcrrkq80000gn/T/ipykernel_78148/4126974325.py:33: DeprecationWarning: 
********************************************************************************
DEPRECATION WARNING:
SmartDataframe will soon be deprecated. Please use df.chat() instead.
********************************************************************************

  df = SmartDataframe(df, config={"llm": llm})
/opt/homebrew/anaconda3/envs/llmApp/lib/python3.10/site-packages/jinja2/runtime.py:303: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return __obj(*args, **kwargs)
/opt/homebrew/anaconda3/envs/llmApp/lib/python3.10/site-packages/jinja2/runtime.py:303: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return __obj(*args, **kwargs)
/opt/homebrew/anaconda3/envs/llmApp/lib/python3.10/site-packages/jinja2/runtime.py:303: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return __obj(*args,

ErrorResponse(type='error', value='Unfortunately, I was not able to get your answer. Please try again.')